In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade pip

# Unsloth and core LLM stack
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Extra deps for inference and metrics
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets sacrebleu nltk "unbabel-comet>=2.2.0" tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8jktpghf/unsloth_83a3ba3379d34cd6a8aa91ee310ba4d6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8jktpghf/unsloth_83a3ba3379d34cd6a8aa91ee310ba4d6
  Resolved https://github.com/unslothai/unsloth.git to commit 3c658a6e1e477990b02b27bf148ee34fb0e24caf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 52.1 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 79.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.6 MB/s  0:00:00
  Created wheel for unsloth: fil

In [ ]:
# Install portalocker for sacrebleu dependency
!pip install portalocker colorama pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 24.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 44.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [pytorch_lightning]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.7 requires entmax<2.0,>=1.1, which is not installed.
unbabel-comet 2.2.7 requires jsonargparse==3.13.1, which is not installed.
unbabel-comet 2.2.7 requires numpy<2.0.0,>=1.20.0, but you have numpy 2.0.2 which is incompatible.
unbabel-comet 2.2.7 requires protobuf<5.0.0,>=4.24.4, but you have protobuf 5.29.5 which is incompatible.
unbabel-comet 2.2.7 requires torchmetrics<0.11.0,>=0.10.2, but you have torchmetrics 1.8.2 which is incompatible.


In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import Dataset
import sacrebleu
import nltk
from nltk.translate.meteor_score import meteor_score
from comet import download_model, load_from_checkpoint
from tqdm.auto import tqdm

nltk.download("wordnet")
nltk.download("omw-1.4")

device = "cuda" if torch.cuda.is_available() else "cpu"
device

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


'cuda'

In [ ]:

ES_PATH = "/content/drive/My Drive/Machine Translation/Project/data/test/all-filtered.es.real.test"
EN_REF_PATH = "/content/drive/My Drive/Machine Translation/Project/data/test/all-filtered.en.real.test"
ES_PATH, EN_REF_PATH


('/content/drive/My Drive/Machine Translation/Project/data/test/all-filtered.es.real.test',
 '/content/drive/My Drive/Machine Translation/Project/data/test/all-filtered.en.real.test')

In [ ]:
MAX_LINES = 5000

with open(ES_PATH, encoding="utf-8") as f_es:
    es_lines = [line.strip() for line in f_es]

with open(EN_REF_PATH, encoding="utf-8") as f_en:
    en_lines = [line.strip() for line in f_en]

print("Total ES lines:", len(es_lines))
print("Total EN lines:", len(en_lines))

n = min(MAX_LINES, len(es_lines), len(en_lines))
es_lines = es_lines[:n]
en_lines = en_lines[:n]

print("Using first", n, "sentence pairs")
print("Example ES:", es_lines[0])
print("Example EN:", en_lines[0])


Total ES lines: 10000
Total EN lines: 10000
Using first 5000 sentence pairs
Example ES: Período de validez después de abierto el envase: 10 horas.
Example EN: Shelf life after first opening the container: 10 hours.


In [ ]:
from unsloth import FastLanguageModel

MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"  # change to your fine tuned model if needed
max_seq_length = 2048
dtype = None          # let Unsloth decide (float16 or bfloat16 on GPU)
load_in_4bit = True   # 4 bit quantization

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = MODEL_NAME,
    max_seq_length  = max_seq_length,
    dtype           = dtype,
    load_in_4bit    = load_in_4bit,
)

FastLanguageModel.for_inference(model)  # enables faster inference
# model.to(device) # This line is not needed for 8-bit bitsandbytes models

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

print("Loaded model on", device)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Loaded model on cuda


In [ ]:
def build_prompt(sentence: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "You are a professional translator. Translate from Spanish to natural idiomatic English."
        },
        {
            "role": "user",
            "content": sentence,
        },
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt


@torch.no_grad()
def translate_batch(sentences, batch_size=32, max_new_tokens=128):
    outputs_all = []
    for start in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[start:start + batch_size]
        prompts = [build_prompt(s) for s in batch]

        encoded = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_seq_length,
        ).to(device)

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        # same length for all after padding
        prompt_len = input_ids.shape[1]

        gen_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

        for i in range(gen_ids.size(0)):
            # slice off the prompt, keep only new tokens
            gen_only_ids = gen_ids[i, prompt_len:]
            text = tokenizer.decode(gen_only_ids, skip_special_tokens=True)
            outputs_all.append(text.strip())

    return outputs_all


In [ ]:
BATCH_SIZE = 32
MAX_NEW_TOKENS = 128

translations = translate_batch(
    es_lines,
    batch_size=BATCH_SIZE,
    max_new_tokens=MAX_NEW_TOKENS,
)

len(translations), translations[0][:200]

  0%|          | 0/157 [00:00<?, ?it/s]

(5000, '"Shelf life after opening the container: 10 hours."')

In [ ]:
OUT_PATH = "/content/drive/My Drive/Machine Translation/Project/all-filtered.es.real.test.llama31.en"

with open(OUT_PATH, "w", encoding="utf-8") as f:
    for line in translations:
        f.write(line.replace("\n", " ").strip() + "\n")

print("Saved translations to", OUT_PATH)

Saved translations to /content/drive/My Drive/Machine Translation/Project/all-filtered.es.real.test.llama31.en


In [ ]:
pip install entmax

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.7 requires jsonargparse==3.13.1, which is not installed.
unbabel-comet 2.2.7 requires numpy<2.0.0,>=1.20.0, but you have numpy 2.0.2 which is incompatible.
unbabel-comet 2.2.7 requires protobuf<5.0.0,>=4.24.4, but you have protobuf 5.29.5 which is incompatible.
unbabel-comet 2.2.7 requires torchmetrics<0.11.0,>=0.10.2, but you have torchmetrics 1.8.2 which is incompatible.


Scoring

In [ ]:
import torch
import sacrebleu
import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from comet import download_model, load_from_checkpoint

nltk.download("punkt")
nltk.download("punkt_tab")  # sometimes needed on newer NLTK versions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
OUT_PATH = "/content/drive/My Drive/Machine Translation/Project/all-filtered.es.real.test.llama31.en"

# Adjust these two if the files are in a different folder/name
REF_PATH = "/content/drive/My Drive/Machine Translation/Project/data/test/all-filtered.en.real.test"
SRC_PATH = "/content/drive/My Drive/Machine Translation/Project/data/test/all-filtered.es.real.test"

# Load system translations
with open(OUT_PATH, encoding="utf-8") as f_sys:
    sys_lines = [line.strip() for line in f_sys]

# Load English references
with open(REF_PATH, encoding="utf-8") as f_ref:
    ref_lines = [line.strip() for line in f_ref]

# (Optional) Load Spanish source for COMET
with open(SRC_PATH, encoding="utf-8") as f_src:
    src_lines = [line.strip() for line in f_src]

print("System lines:", len(sys_lines))
print("Reference lines:", len(ref_lines))
print("Source lines:", len(src_lines))

# Make sure lengths match (we only use aligned prefix)
n = min(len(sys_lines), len(ref_lines), len(src_lines))
sys_lines = sys_lines[:n]
ref_lines = ref_lines[:n]
src_lines = src_lines[:n]

print("Using", n, "aligned sentence pairs")
print("Example REF:", ref_lines[0])
print("Example SYS:", sys_lines[0])


System lines: 5000
Reference lines: 10000
Source lines: 10000
Using 5000 aligned sentence pairs
Example REF: Shelf life after first opening the container: 10 hours.
Example SYS: "Shelf life after opening the container: 10 hours."


In [ ]:
# SacreBLEU expects: sys = list[str], refs = list[list[str]]
bleu = sacrebleu.corpus_bleu(sys_lines, [ref_lines])
chrf = sacrebleu.corpus_chrf(sys_lines, [ref_lines])

print(f"BLEU: {bleu.score:.2f}")
print(f"chrF: {chrf.score:.2f}")

BLEU: 33.36
chrF: 65.10


In [ ]:
meteor_scores = [
    meteor_score(
        [word_tokenize(ref)],   # tokenized reference (inside a list)
        word_tokenize(hyp)      # tokenized hypothesis
    )
    for ref, hyp in zip(ref_lines, sys_lines)
]

meteor_avg = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR: {meteor_avg:.4f}")

Average METEOR: 0.6423


In [ ]:
# Load COMET model (cached after first download)
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

# Build COMET input data
comet_data = [
    {
        "src": src,   # Spanish source
        "mt": hyp,    # model translation
        "ref": ref,   # English reference
    }
    for src, hyp, ref in zip(src_lines, sys_lines, ref_lines)
]

device = "cuda" if torch.cuda.is_available() else "cpu"
gpus = 1 if device == "cuda" else 0

comet_output = comet_model.predict(
    comet_data,
    batch_size=16,
    gpus=gpus,
)

print("COMET system score:", comet_output.system_score)
print("First 5 segment scores:", comet_output.scores[:5])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_

COMET system score: 0.8304899024009704
First 5 segment scores: [0.9055176377296448, 0.7112483382225037, 0.9082013964653015, 0.9045613408088684, 0.8010120391845703]


In [ ]:
print("Evaluation on", len(sys_lines), "sentence pairs")
print(f"BLEU:   {bleu.score:.2f}")
print(f"chrF:   {chrf.score:.2f}")
print(f"METEOR: {meteor_avg:.4f}")
print(f"COMET:  {comet_output.system_score:.4f}")

Evaluation on 5000 sentence pairs
BLEU:   33.36
chrF:   65.10
METEOR: 0.6423
COMET:  0.8305
